## Solved (EDIT) 

it was never an issue with timing out, I was unaware I could get errorlogs from failed submissions (new to this). Should note that running locally the `lux-ai-2021` command doesn't give the 60s for the first submission I don't think. so use `--maxtime` to set the buffer higher if necessary. 

This Notebook is prepared to attempt to debug the Issue raised in the discussion, https://www.kaggle.com/c/lux-ai-2021/discussion/277324, it's a notebook based of Imitation learning notebook, https://www.kaggle.com/shoheiazuma/lux-ai-with-imitation-learning. 

The Issue at hand is that importing `Tensorflow` or `tf.keras.models.load_model` takes over 3s, which appears to cause a timeout on the first turn, implying that the 60s allowed for imports on the first call isn't being invoked


If Anyone had experienced something similar. would appreciate any comments on how to fix this issue

In [ ]:
!pip install tensorflow==2.3
#!git clone https://github.com/Lux-AI-Challenge/Lux-Design-2021

# create simple model and prepare data

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
from collections import deque
import json 
from pathlib import Path
import os
import random
from tqdm.notebook import tqdm
import sys


#preprocessing SAZUMA Kaggle notebook
def to_label(action):
    strs = action.split(' ')
    unit_id = strs[1]
    if strs[0] == 'm':
        label = {'c': None, 'n': 0, 's': 1, 'w': 2, 'e': 3}[strs[2]]
    elif strs[0] == 'bcity':
        label = 4
    else:
        label = None
    return unit_id, label


def depleted_resources(obs):
    for u in obs['updates']:
        if u.split(' ')[0] == 'r':
            return False
    return True

"""
create dataset from the episodes, 
samples = list of tuples [(observation_id, unit_id (e.g worker id), label/action (e.g move West)), ...]
"""


def create_dataset_from_json(episode_dir, team_name='Toad Brigade'):
    obses = {}
    samples = []
    append = samples.append
    #prepend with 5.json to select only certain epsisodes for memory purpose
    #full version should use tf.Dataset and *.json to take all the episodes
    episodes = [path for path in Path(episode_dir).glob('*5.json') if 'replay' not in path.name]
    print(episodes[1])
    for filepath in tqdm(episodes):
        with open(filepath) as f:
            json_load = json.load(f)
        ep_id = json_load['info']['EpisodeId']
        index = np.argmax([r or 0 for r in json_load['rewards']])
        if json_load['info']['TeamNames'][index] != team_name:
            continue

        for i in range(len(json_load['steps'])-1):
            if json_load['steps'][i][index]['status'] == 'ACTIVE':
                actions = json_load['steps'][i+1][index]['action']
                obs = json_load['steps'][i][0]['observation']

                if depleted_resources(obs):
                    break

                obs['player'] = index
                obs = dict([
                    (k,v) for k,v in obs.items()
                    if k in ['step', 'updates', 'player', 'width', 'height']
                ])
                obs_id = f'{ep_id}_{i}'
                obses[obs_id] = obs

                for action in actions:
                    unit_id, label = to_label(action)
                    if label is not None:
                        append((obs_id, unit_id, label))

    return obses, samples

##Test above funcs
obses, samples = create_dataset_from_json('../input/lux-ai-episodes/')
print(type(samples[0]))
print(samples[0])
count = 0
for _,value in obses.items():
		print(value['updates'])
		if count > 0:
				break
		count +=1 


labels = [sample[-1] for sample in samples]
actions = ['North', 'South', 'West', 'East', 'Build_City']
for value, count in zip(*np.unique(labels, return_counts=True)):
		print(f'{actions[value]:^5}: {count:>3}')

print(set(labels))

# Input for Neural Network
def make_input(obs, unit_id):
    width, height = obs['width'], obs['height']
    x_shift = (32 - width) // 2
    y_shift = (32 - height) // 2
    cities = {}
    
    b = np.zeros((20, 32, 32), dtype=np.float32)
    
    for update in obs['updates']:
        strs = update.split(' ')
        input_identifier = strs[0]
        
        if input_identifier == 'u':
            x = int(strs[4]) + x_shift
            y = int(strs[5]) + y_shift
            wood = int(strs[7])
            coal = int(strs[8])
            uranium = int(strs[9])
            if unit_id == strs[3]:
                # Position and Cargo
                b[:2, x, y] = (
                    1,
                    (wood + coal + uranium) / 100
                )
            else:
                # Units
                team = int(strs[2])
                cooldown = float(strs[6])
                idx = 2 + (team - obs['player']) % 2 * 3
                b[idx:idx + 3, x, y] = (
                    1,
                    cooldown / 6,
                    (wood + coal + uranium) / 100
                )
        elif input_identifier == 'ct':
            # CityTiles
            team = int(strs[1])
            city_id = strs[2]
            x = int(strs[3]) + x_shift
            y = int(strs[4]) + y_shift
            idx = 8 + (team - obs['player']) % 2 * 2
            b[idx:idx + 2, x, y] = (
                1,
                cities[city_id]
            )
        elif input_identifier == 'r':
            # Resources
            r_type = strs[1]
            x = int(strs[2]) + x_shift
            y = int(strs[3]) + y_shift
            amt = int(float(strs[4]))
            b[{'wood': 12, 'coal': 13, 'uranium': 14}[r_type], x, y] = amt / 800
        elif input_identifier == 'rp':
            # Research Points
            team = int(strs[1])
            rp = int(strs[2])
            b[15 + (team - obs['player']) % 2, :] = min(rp, 200) / 200
        elif input_identifier == 'c':
            # Cities
            city_id = strs[2]
            fuel = float(strs[3])
            lightupkeep = float(strs[4])
            cities[city_id] = min(fuel / lightupkeep, 10) / 10
    
    # Day/Night Cycle
    b[17, :] = obs['step'] % 40 / 40
    # Turns
    b[18, :] = obs['step'] / 360
    # Map Size
    b[19, x_shift:32 - x_shift, y_shift:32 - y_shift] = 1

    return b


def create_model(n_features, n_actions):
	model = Sequential()
	model.add(Conv2D(32, (3, 3), input_shape = (20,32,32)))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(tf.keras.layers.Flatten())
	model.add(tf.keras.layers.Dense(20, activation = 'linear'))
	model.add(tf.keras.layers.Dense(n_actions, activation = 'softmax'))
	model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = Adam(learning_rate = .001))
	return model
from matplotlib import pyplot

def train_model(model, inputs, output, batch_size, epochs, val_in, val_out):
		mc = tf.keras.callbacks.ModelCheckpoint('best_model', monitor = 'val_loss', save_best_only = True, save_weights_only = True)
		history = model.fit(inputs, output, batch_size = batch_size, epochs = epochs, validation_data = (val_in, val_out), verbose = 1)
		tf.keras.models.save_model(model, "model")
		pyplot.subplot(211)
		pyplot.title('Loss ')
		pyplot.plot(history.history['loss'],label='train')
		pyplot.plot(history.history['val_loss'],label='test')
		pyplot.legend()

		pyplot.subplot(212)
		pyplot.title('Accuracy ')
		pyplot.plot(history.history['accuracy'],label='train')
		pyplot.plot(history.history['val_accuracy'],label='test')
		pyplot.legend()
		pyplot.show()

from sklearn.model_selection import train_test_split

train_samples, val_samples = train_test_split(samples, test_size=.15, random_state = 8, stratify = labels)

def prep(samples):

	input_matrix = []
	output = []
	for (obs_id, unit_id, action) in samples:
			obs = obses[obs_id]
			input_matrix.append(make_input(obs, unit_id))
			out = []
			for i in range(0,5):
					if i == action:
							out.append(1)
					else:
							out.append(0)
			output.append(np.array(out))
	return input_matrix, output	

train_x, train_y = prep(train_samples)
val_x, val_y = prep(val_samples)




# Train Model

In [ ]:

model = create_model(20, 5)
history = train_model(model, np.array(train_x), np.array(train_y), 64, 8, np.array(val_x), np.array(val_y))


# install kaggle-environments to run the games

In [ ]:
!pip install kaggle-environments -U > /dev/null 2>&1
!cp -r ../input/lux-ai-2021/* .


# Overwrite agent.py and main.py

In [ ]:
%%writefile agent.py

import time
t1 = time.time()
import math, sys
from lux.game import Game
import numpy as np
from tensorflow.keras.models import load_model
import os


game_state = None
#print(time.time() - t1, file=sys.stderr)


# Input for Neural Network
def make_input(obs, unit_id):
    width, height = obs['width'], obs['height']
    x_shift = (32 - width) // 2
    y_shift = (32 - height) // 2
    cities = {}
    
    b = np.zeros((20, 32, 32), dtype=np.float32)
    
    for update in obs['updates']:
        strs = update.split(' ')
        input_identifier = strs[0]
        
        if input_identifier == 'u':
            x = int(strs[4]) + x_shift
            y = int(strs[5]) + y_shift
            wood = int(strs[7])
            coal = int(strs[8])
            uranium = int(strs[9])
            if unit_id == strs[3]:
                # Position and Cargo
                b[:2, x, y] = (
                    1,
                    (wood + coal + uranium) / 100
                )
            else:
                # Units
                team = int(strs[2])
                cooldown = float(strs[6])
                idx = 2 + (team - obs['player']) % 2 * 3
                b[idx:idx + 3, x, y] = (
                    1,
                    cooldown / 6,
                    (wood + coal + uranium) / 100
                )
        elif input_identifier == 'ct':
            # CityTiles
            team = int(strs[1])
            city_id = strs[2]
            x = int(strs[3]) + x_shift
            y = int(strs[4]) + y_shift
            idx = 8 + (team - obs['player']) % 2 * 2
            b[idx:idx + 2, x, y] = (
                1,
                cities[city_id]
            )
        elif input_identifier == 'r':
            # Resources
            r_type = strs[1]
            x = int(strs[2]) + x_shift
            y = int(strs[3]) + y_shift
            amt = int(float(strs[4]))
            b[{'wood': 12, 'coal': 13, 'uranium': 14}[r_type], x, y] = amt / 800
        elif input_identifier == 'rp':
            # Research Points
            team = int(strs[1])
            rp = int(strs[2])
            b[15 + (team - obs['player']) % 2, :] = min(rp, 200) / 200
        elif input_identifier == 'c':
            # Cities
            city_id = strs[2]
            fuel = float(strs[3])
            lightupkeep = float(strs[4])
            cities[city_id] = min(fuel / lightupkeep, 10) / 10
    
    # Day/Night Cycle
    b[17, :] = obs['step'] % 40 / 40
    # Turns
    b[18, :] = obs['step'] / 360
    # Map Size
    b[19, x_shift:32 - x_shift, y_shift:32 - y_shift] = 1

    return b

def in_city(pos):    
    try:
        city = game_state.map.get_cell_by_pos(pos).citytile
        return city is not None and city.team == game_state.id
    except:
        return False


def call_func(obj, method, args=[]):
    return getattr(obj, method)(*args)


unit_actions = [('move', 'n'), ('move', 's'), ('move', 'w'), ('move', 'e'), ('build_city',)]

def get_action(policy, unit, dest):
    for label in np.argsort(policy)[::-1]:
        act = unit_actions[label]
        pos = unit.pos.translate(act[-1], 1) or unit.pos
        if pos not in dest or in_city(pos):
            return call_func(unit, *act), pos 
            
    return unit.move('c'), unit.pos
#print(f"{time.time() - t1} test")
print(time.time() - t1, file=sys.stderr)
path = '/kaggle_simulations/agent' if os.path.exists('/kaggle_simulations') else '.'
print(f"path is {path}", file=sys.stderr)
model = load_model(f"{path}/model")
def agent(observation, configuration):
	global game_state
	print("agent Called", file=sys.stderr)

	
	### Do not edit ###
	if observation["step"] == 0:
		game_state = Game()
		game_state._initialize(observation["updates"])
		game_state._update(observation["updates"][2:])
		game_state.id = observation.player
	else:
		game_state._update(observation["updates"])
    
	actions = []

    ### AI Code goes down here! ### 
	player = game_state.players[observation.player]
	opponent = game_state.players[(observation.player + 1) % 2]
	width, height = game_state.map.width, game_state.map.height

	
	"""when we get here we should have trained the DQN on games so when we actually play 
	we loop through the available units (player, citytiles, Carts) and call model(cart/worker/citytile)
	to get which action each tile should take ????"""
#	t1 = time.time()
	unit_count = len(player.units)
	for city in player.cities.values():
			for citytile in city.citytiles:
					if citytile.can_act():
							if unit_count < player.city_tile_count:
									actions.append(citytile.build_worker())
									unit_count += 1
							elif not player.researched_uranium():
									actions.append(citytile.research())
									player.research_points += 1

	
	dest = []
	for unit in player.units:
		if unit.can_act() and (game_state.turn % 40 < 30 or not in_city(unit.pos)):
				state = make_input(observation, unit.id)
				state1 = np.reshape(state, (1,20,32,32))
				policy = model.predict(state1)
				tm2 = time.time()
				action, pos = get_action(policy[0], unit, dest)
				actions.append(action)
				dest.append(pos)

	return actions




Run the game using `make`, this runs perfectly fine with no timeouts.

In [ ]:
%%writefile main.py

import time
t1 = time.time()
from typing import Dict
import sys
from agent import agent
print(f"{time.time() - t1} time to load in main", file=sys.stderr)
if __name__ == "__main__":
    
  def read_input():
    """
    Reads input from stdin
    """
    try:
      return input()
    except EOFError as eof:
      raise SystemExit(eof)
  step = 0
  class Observation(Dict[str, any]):
    def __init__(self, player=0) -> None:
      self.player = player
      # self.updates = []
      # self.step = 0
  observation = Observation()
  observation["updates"] = []
  observation["step"] = 0
  player_id = 0
  count = 0
  while True:
    t2 = time.time()
    inputs = read_input()
    observation["updates"].append(inputs)
    if inputs == "D_DONE":
      if step == 0:  # the codefix
        player_id = int(observation["updates"][0])
        observation.player = player_id
        observation["player"] = player_id
        observation["width"], observation["height"] = map(int, observation["updates"][1].split())
      print(f"{time.time() - t2} time from start of loop til agent", file=sys.stderr)
      t3 = time.time()
      if count < 2:
        print(f"{time.time() - t1}: time to {count} agent call", file=sys.stderr)
        count += 1
      actions = agent(observation, None)
      print(f"{time.time() - t3}: Time to get agents actions", file=sys.stderr)
      observation["updates"] = []
      step += 1
      observation["step"] = step
      print(",".join(actions))
      print("D_FINISH")
      t1 = time.time()


In [ ]:
from kaggle_environments import make

env = make("lux_ai_2021", configuration={"width": 24, "height": 24, "loglevel": 2, "annotations": True}, debug=True)
steps = env.run(['agent.py', 'agent.py'])
env.render(mode="ipython", width=800, height=600)

Uncomment and run below code to see time it takes to import files in `main.py` on the first call. *will run indefinitely so must interrupt manually to stop it*

In [ ]:
#!python3 main.py

create submission file

In [ ]:
!tar -czvf submission.tar.gz *

Trying to use `lux-ai-2021` command to help debugging, unsure how to use in Kaggle Notebook

In [ ]:
#!lux-ai-2021 ./Lux-Design-2021/kits/python/simple/main.py ./Lux-Design-2021/kits/python/simple/main.py 